# Objectives of the Assignment 

1. 99.4% validation accuracy
2. Less than 20k Parameters
3. You can use anything from above you want. 
4. Less than 20 Epochs
5. Have used BN, Dropout, a Fully connected layer, have used GAP. 
6. To learn how to add different things we covered in this session, 

you can refer to this code: https://www.kaggle.com/enwei26/mnist-digits-pytorch-cnn-99 DONT COPY ARCHITECTURE, JUST LEARN HOW TO INTEGRATE THINGS LIKE DROPOUT, BATCHNORM, ET

In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
class Net(nn.Module):

  def __init__(self):
    super(Net,self).__init__()

    self.conv1 = nn.Conv2d(1 , 12, 5) # Input : 1x28x28 , Output : 12x24x24 , RF: 5 , Convolution Operation 
    self.batch1 = nn.BatchNorm2d(12) # Batch Normalization 
    self.pool1 = nn.MaxPool2d(2,2) # Input : 12x24x24 , Output : 12x12x12 , RF: 9
    self.dropout1 = nn.Dropout(0.25) # Adding Dropouts
   
    self.conv2 = nn.Conv2d(12, 24, 3) # Input : 12x12x12 , Output : 24x10x10 , RF: 11 ,  Convolution Operation 
    self.batch2 = nn.BatchNorm2d(24) # Batch Normalization 
    self.pool2 = nn.MaxPool2d(2,2) # Input : 24x10x10 , Output : 24x5x5 , RF: 13 
    self.dropout2 = nn.Dropout(0.25) # Adding Dropouts 
    
    self.conv3 = nn.Conv2d(24, 30, 3) # Input : 24x5x5 , Output : 30x3x3, RF: 15 ,  Convolution Operation 
    self.batch3 = nn.BatchNorm2d(30) # Batch Normalization 
    
    self.conv4 = nn.Conv2d(30, 10, 3) # Input : 30x3x3 , Output : 10x1x1 , RF : 17 ,  Convolution Operation 
  
  def forward(self, x):
    ## Block 1 
    x = self.pool1(self.batch1(F.relu(self.conv1(x))))
    x = self.dropout1(x)

    ## Block 2
    x = self.pool2(self.batch2(F.relu(self.conv2(x))))
    x = self.dropout2(x)

    ## Block 3
    x = self.batch3(F.relu(self.conv3(x)))
    
    ## Block 4
    x = self.conv4(x)

    x = x.view(-1, 10)
    return F.log_softmax(x)

## Intialising the GPU for creating the Convolutional Neural Network 

In [3]:

!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 12, 24, 24]             312
       BatchNorm2d-2           [-1, 12, 24, 24]              24
         MaxPool2d-3           [-1, 12, 12, 12]               0
           Dropout-4           [-1, 12, 12, 12]               0
            Conv2d-5           [-1, 24, 10, 10]           2,616
       BatchNorm2d-6           [-1, 24, 10, 10]              48
         MaxPool2d-7             [-1, 24, 5, 5]               0
           Dropout-8             [-1, 24, 5, 5]               0
            Conv2d-9             [-1, 30, 3, 3]           6,510
      BatchNorm2d-10             [-1, 30, 3, 3]              60
           Conv2d-11             [-1, 10, 1, 1]           2,710
Total params: 12,280
Trainable params: 12,280
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/ba

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:33: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


## Created data that has to be loaded in Bactches 

In [4]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


## Creating a Training Function 

In [5]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

### Model Performance and Evaluation 

In [6]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=0.08374646306037903 batch_id=468: 100%|██████████| 469/469 [00:42<00:00, 11.12it/s]



Test set: Average loss: 0.0546, Accuracy: 9833/10000 (98%)



loss=0.08233388513326645 batch_id=468: 100%|██████████| 469/469 [00:42<00:00, 11.08it/s]



Test set: Average loss: 0.0386, Accuracy: 9873/10000 (99%)



loss=0.09380369633436203 batch_id=468: 100%|██████████| 469/469 [00:43<00:00, 10.90it/s]


...


Test set: Average loss: 0.0187, Accuracy: 9937/10000 (99%)



loss=0.055500756949186325 batch_id=468: 100%|██████████| 469/469 [00:43<00:00, 10.86it/s]



Test set: Average loss: 0.0192, Accuracy: 9939/10000 (99%)



loss=0.007333959918469191 batch_id=468: 100%|██████████| 469/469 [00:42<00:00, 11.05it/s]



Test set: Average loss: 0.0186, Accuracy: 9942/10000 (99%)

